In [1]:
import pandas 

zip_codes_by_city = pandas.read_csv("./free-zipcode-database-Primary.csv")
new_york_zip_codes = zip_codes_by_city[zip_codes_by_city["City"].isin(["BRONX", "BROOKLYN","NEW YORK","STATEN ISLAND", "QUEENS"])]
new_york_zip_codes = new_york_zip_codes[
    (new_york_zip_codes["Lat"] >= 40) &
    (new_york_zip_codes["Lat"] <= 41) &
    (new_york_zip_codes["Long"] <= -73) &
    (new_york_zip_codes["Long"] >= -75)
    ]
new_york_zip_codes["geolocation"]= list(zip(new_york_zip_codes.loc[:,"Lat"], -abs(new_york_zip_codes.loc[:,"Long"])))
zip_code_locations = new_york_zip_codes["geolocation"].tolist()

In [2]:
import gmaps
import gmaps.datasets
import googlemaps
api_key = open("../shared/shared_api_key.txt").readline()
gmaps.configure(api_key=api_key) # Your Google API key
google_map_api = googlemaps.Client(key=api_key)


In [9]:
from datetime import datetime
import time
import math
search_radius = 1609.34 #meters
weighted_heatmap_data = []
last_location = ()
locations = []
num_locations = 0
for index, row in new_york_zip_codes.iterrows():

    if num_locations > 500:
        break
    if last_location == (row["Lat"], row["Long"]) or math.isnan(row["Lat"]):
        continue
    num_locations += 1

    print(num_locations)
    print(row["Lat"])
    print(row["Long"])

    last_location = (row["Lat"], row["Long"])
    thai_restaurants_nearby = google_map_api.places_nearby(location=last_location,keyword='thai restaurants near ', radius=2*search_radius)
    number_thai_restaurants = len(thai_restaurants_nearby["results"]) + 1
    locations.append(last_location)
    weighted_heatmap_data.append((row["Lat"],row["Long"], number_thai_restaurants))
    time.sleep(.05)
    

1
40.84
-73.87
2
40.71
-73.99
3
40.77
-73.95
4
40.71
-73.99
5
40.76
-73.96
6
40.71
-73.99
7
40.77
-73.95
8
40.71
-73.99
9
40.64
-73.94
10
40.58
-74.14
11
40.71
-74.0
12
40.71
-74.01
13
40.75
-73.99
14
40.71
-74.0
15
40.77
-73.98
16
40.71
-74.0
17
40.75
-73.99
18
40.71
-74.0
19
40.76
-73.98
20
40.71
-74.0
21
40.69
-73.98
22
40.68
-73.99
23
40.69
-73.99
24
40.69
-73.98


In [8]:
m = gmaps.Map()
thai_layer = gmaps.WeightedHeatmap(data= weighted_heatmap_data)
thai_layer.max_intensity = 25
thai_layer.point_radius = 25
thai_layer.opacity = .6
m.add_layer(thai_layer)
markers = gmaps.marker_layer(locations)
m.add_layer(markers)

m


In [5]:
len(new_york_zip_codes)

256